In [1]:
from pathlib import Path
from datetime import UTC

import pandas as pd
import numpy as np

Carrega dados.

In [2]:
data_dir = Path('data')
br_path = data_dir / 'bitrate_train.csv'
rt_path = data_dir / 'rtt_train.csv'

br = pd.read_csv(br_path)
br['data_index'] = pd.to_datetime(br['timestamp'], unit='s')
br = br.set_index('data_index')
br = br.sort_index()

rt = pd.read_csv(rt_path)
rt['data_index'] = pd.to_datetime(rt['timestamp'], unit='s')
rt = rt.set_index('data_index')
rt = rt.sort_index()

def get_by_states(data, states):
    return data[((data['client'] == states[0] )& (data['server'] == states[1]))]

Agrupa dados por rajada e tira a média do bitrate. Associa os dados de latência a um intervalo de tempo associado a cada dado de bitrate. 

In [3]:
dfs = []
for g in br.groupby(['client', 'server']):
    rt2 = rt[((rt['client'] == g[0][0]) & (rt['server'] == g[0][1]))]
    df = g[1]
    df = df.groupby(np.arange(len(df))//15).agg({'timestamp': 'first', 'bitrate': 'mean', 'client': 'first', 'server': 'first'})

    df['min'] = df['timestamp'] - 150
    df['max'] = df['timestamp'] + 150
    df.loc[:, ['rtt']] = 0.0
    df.loc[:, ['rtt_count']] = 0
    df.loc[:, ['rtt_ts']] = 0
    df.index = pd.IntervalIndex.from_arrays(df['min'],df['max'],closed='left')
    
    for row in rt2.iterrows():
        rt_timestamp = row[1].timestamp
        rt_rtt = row[1].rtt
        try:
            df_loc = df.index.get_loc(rt_timestamp)
            df.iloc[df_loc, 6] += rt_rtt
            df.iloc[df_loc, 7] += 1
            df.iloc[df_loc, 8] = rt_timestamp
        except:
            pass
    dfs.append(df)
df = pd.concat(dfs)
df

,timestamp,bitrate,client,server,min,max,rtt,rtt_count,rtt_ts
"[1717718765, 1717719065)",1717718915,308724.066667,ba,ce,1717718765,1717719065,12.52,1,1717718941
"[1717719065, 1717719365)",1717719215,306280.266667,ba,ce,1717719065,1717719365,12.59,1,1717719217
"[1717719365, 1717719665)",1717719515,349834.933333,ba,ce,1717719365,1717719665,0.00,0,0
"[1717719666, 1717719966)",1717719816,315504.000000,ba,ce,1717719666,1717719966,24.98,2,1717719841
"[1717719966, 1717720266)",1717720116,348891.466667,ba,ce,1717719966,1717720266,12.48,1,1717720185
...,...,...,...,...,...,...,...,...,...
"[1718162436, 1718162736)",1718162586,302007.133333,rj,pi,1718162436,1718162736,42.06,1,1718162447
"[1718162736, 1718163036)",1718162886,290014.266667,rj,pi,1718162736,1718163036,42.00,1,1718162968
"[1718163036, 1718163336)",1718163186,295756.333333,rj,pi,1718163036,1718163336,0.00,0,0
"[1718163336, 1718163636)",1718163486,283097.400000,rj,pi,1718163336,1718163636,41.99,1,1718163504


In [4]:
df = df[df['rtt'] != 0]
df.loc[:, 'rtt'] = df['rtt'] / df['rtt_count']
df = df.loc[:, ['timestamp', 'client', 'server', 'bitrate', 'rtt']]
df['data_index'] = pd.to_datetime(df['timestamp'], unit='s')
df = df.set_index('data_index')
df

,timestamp,client,server,bitrate,rtt
data_index,,,,,
2024-06-07 00:08:35,1717718915,ba,ce,308724.066667,12.52
2024-06-07 00:13:35,1717719215,ba,ce,306280.266667,12.59
2024-06-07 00:23:36,1717719816,ba,ce,315504.000000,12.49
2024-06-07 00:28:36,1717720116,ba,ce,348891.466667,12.48
2024-06-07 00:33:36,1717720416,ba,ce,310630.933333,12.49
...,...,...,...,...,...
2024-06-12 03:13:06,1718161986,rj,pi,309949.200000,41.96
2024-06-12 03:18:06,1718162286,rj,pi,286421.200000,42.00
2024-06-12 03:23:06,1718162586,rj,pi,302007.133333,42.06


Normaliza os dados entre (1e-6, 1)

In [5]:
def min_max(x, range=(1e-6,1)):
    x_std = (x - x.min(axis=0)) / (x.max(axis=0) - x.min(axis=0))
    x_scaled = x_std * (range[1] - range[0]) + range[0]
    return x_scaled

df.loc[:, 'bitrate'] = df.loc[:, ['bitrate']].apply(min_max)
df.loc[:, 'rtt'] = df.loc[:, ['rtt']].apply(min_max)

#df.loc[:, 'bitrate'] = df[['client', 'server', 'bitrate']].groupby(['client', 'server']).transform(min_max)
#df.loc[:, 'rtt'] = df[['client', 'server', 'rtt']].groupby(['client', 'server']).transform(min_max)

df['qoe'] = df['bitrate']/df['rtt']
df

,timestamp,client,server,bitrate,rtt,qoe
data_index,,,,,,
2024-06-07 00:08:35,1717718915,ba,ce,0.673730,0.036022,18.703104
2024-06-07 00:13:35,1717719215,ba,ce,0.668201,0.036470,18.322124
2024-06-07 00:23:36,1717719816,ba,ce,0.689071,0.035831,19.231304
2024-06-07 00:28:36,1717720116,ba,ce,0.764617,0.035767,21.377827
2024-06-07 00:33:36,1717720416,ba,ce,0.678045,0.035831,18.923572
...,...,...,...,...,...,...
2024-06-12 03:13:06,1718161986,rj,pi,0.676502,0.224116,3.018543
2024-06-12 03:18:06,1718162286,rj,pi,0.623266,0.224371,2.777834
2024-06-12 03:23:06,1718162586,rj,pi,0.658532,0.224754,2.930006


In [6]:
df.to_csv('data/dados.csv', index=False)